In [1]:
# Dependencies and Setup
import gmaps
import pandas as pd
import os
import matplotlib.pyplot as plt
import requests
from config import api_key
import random 
import numpy as np
import requests
import json

# Google developer API key
from googleconfig import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

# File to Load 
cityResult = "Resources/cityWeatherResult.csv"

# Read cities result and put them into Panda dataframe
cityResult_df = pd.read_csv(cityResult)

cityResult_df

,Country,City,Latitude,Longitude,Hemisphere,Processed,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,am,agarak,40.307778,44.284722,N,Y,40.48,70,100,6.46
1,am,aghavnatun,40.233611,44.251389,N,Y,28.40,86,75,2.30
2,am,mrgashen,40.285000,44.542222,N,Y,28.40,86,75,2.30
3,ao,caconda,-13.733333,15.066667,S,Y,58.62,82,100,2.37
4,at,bad hofgastein,47.166667,13.100000,N,Y,34.74,98,86,1.01
...,...,...,...,...,...,...,...,...,...,...
595,us,culpeper,38.473056,-77.996944,N,Y,29.55,40,1,17.27
596,ws,pata,-13.916667,-171.983333,S,Y,71.11,83,81,9.04
597,za,ekangala,-25.696193,28.749183,S,Y,67.03,88,100,2.30
598,za,theunissen,-28.410979,26.701068,S,Y,59.27,83,26,10.54


In [2]:
gmaps.configure(api_key=gkey)

# Store 'Latitude' and 'Longitude' into coordinates 
coordinates = cityResult_df[["Latitude", "Longitude"]].astype(float)

# Convert Humdity to float and store
# HINT: be sure to handle NaN values
cityResult_df = cityResult_df.dropna()
humidity_percentage = cityResult_df["Humidity (%)"].astype(float)

# Create a humdity % Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_percentage, dissipating=False, max_intensity=100,point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [3]:
# Create column to store closing hotel name
cityResult_df["Closest Hotel Name"] = ""

# find the closest Hotel for each ideal city within 5 kilometres radius
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": gkey
}

# Use the search term: "Hotel" and our lat/lng
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in cityResult_df.iterrows():
       #Only process hotel info for Ideal cities where their Temp > 70 and < 80, wind speed < 10, cloudiness = 0 and humidity < 70
       if (row['Temperature (F)'] > 70) & (row['Temperature (F)'] < 80) & (row['Wind Speed (mph)'] < 10) & (row['Cloudiness (%)'] == 0) & (row['Humidity (%)'] < 70):
            # get lat, lng from df
            lat = row["Latitude"]
            lng = row["Longitude"]

            # change location each iteration while leaving original params in place
            params["location"] = f"{lat},{lng}"

            # make request 
            hotel_address = requests.get(base_url, params=params)
            
            #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
            #     print(name_address.url)

            # convert to json
            hotel_address = hotel_address.json()
            # print(json.dumps(name_address, indent=4, sort_keys=True))

            # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
            try:
                cityResult_df.loc[index, "Closest Hotel Name"] = hotel_address["results"][0]["name"]
            except (KeyError, IndexError):
                print("Missing field/result... skipping.")


Missing field/result... skipping.


In [5]:
hotelCityDf = cityResult_df[(cityResult_df['Closest Hotel Name'] !="")]

hotelCityDf
#hotelName = cityResult_df["Closest Hotel Name"].tolist()


,Country,City,Latitude,Longitude,Hemisphere,Processed,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Closest Hotel Name
89,co,silvania,4.403671,-74.386700,N,Y,70.57,68,0,5.75,Primavera Azul Glamping
185,gt,el palmar,14.650000,-91.583333,N,Y,70.70,65,0,6.91,Hotel y Bungalows El Jardín
293,ml,kayes,14.450000,-11.433333,N,Y,71.15,17,0,4.29,Hôtel Kamankole Palace
521,tg,sotouboua,8.566667,0.983333,N,Y,77.00,33,0,2.62,Auberge le Verger


In [6]:
# Convert Hotel Names to list
hotelName = hotelCityDf["Closest Hotel Name"].tolist()
# Convert City Names to list
hotelCity = hotelCityDf["City"].tolist()
# Convert Country to list
hotelCountry = hotelCityDf["Country"].tolist()
hotelCoordinates = hotelCityDf[["Latitude", "Longitude"]].astype(float)


# Create hotel symbol layer
hotelLayer = gmaps.symbol_layer(
    hotelCoordinates, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=[f"<b>Hotel name:</b> <br> {hotel} <br> <b>City:</b> <br> {city}  <br> <b>Country:</b> <br> {country} " for hotel,city,country in zip(hotelName,hotelCity,hotelCountry)]
)

# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotelLayer)
fig

Figure(layout=FigureLayout(height='420px'))